In [1]:
import warnings
import os
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI


warnings.filterwarnings("ignore", category=DeprecationWarning) 
load_dotenv()
openAIApiKey = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(api_key=openAIApiKey)

# Agent

In [2]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

embeddings = OpenAIEmbeddings(api_key=openAIApiKey)
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

e:\CodeProjects\notebooks\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv() 
tavilyKey = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(tavily_api_key=tavilyKey)

ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

In [ ]:
tools = [retriever_tool, search]

In [ ]:
from langchain_community.chat_models import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...


Authentication failed for https://api.smith.langchain.com/runs/e88452c2-666f-4db4-adc3-31bbe6227552. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/e88452c2-666f-4db4-adc3-31bbe6227552', '{"detail":"Invalid auth"}')
Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Invalid auth"}')


LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine which parts of the software have been exercised during testing. It can provide insights into the effectiveness of the test suite and identify areas that need additional testing.

4. Test Prioritization: LangSmith can prioritize test cases based on their likelihood of finding bugs or their i

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine which parts of the software have been exercised during testing. It can provide insights into the effectiveness of the test suite and identify areas that need additional testing.\n\n4. Test Prioritization: LangSmith can prio

In [ ]:
agent_executor.invoke({"input": "I want to go deeper into neuroscience. What is the current state of Artifical Synapse? What are main patent in that field?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'current state of Artificial Synapse'}`


[{'url': 'https://www.nature.com/articles/s41378-023-00487-2', 'content': 'on the current development of artificial synapse properties excited by electrical, optical, mechanical, and hybrid  Recent progress in three-terminal artificial synapses based on 2D materials: from mechanisms to applications  Scott T. Keene, Claudia Lubrano, â€¦ Francesca Santoro An artificial synapse based on molecular junctions  biological neural behaviors129. Finally, the challenges and future perspectives of artificial synapses are outlined.At present, research on three- and multiple-terminal synaptic transistors is still preliminary, and neuromorphic transistors have mainly focused on four types: synapses based on floating...'}, {'url': 'https://hai.stanford.edu/news/what-expect-ai-2024', 'content': 'What to Expect in AI in 2024  not only to foster a vibrant AI ecosystem 

{'input': 'I want to go deeper into neuroscience. What is the current state of Artifical Synapse? What are main patent in that field?',
 'output': "The current state of Artificial Synapse is still in its preliminary stages. Research is mainly focused on three- and multiple-terminal synaptic transistors. There are four main types of synapses being studied: those based on electrical, optical, mechanical, and hybrid properties. The development of artificial synapses is crucial for the construction of biocompatible electronic devices towards artificial intelligence systems. Some of the applications of artificial synapses include various sensory systems, bioplastic bionics, and logical transformation [^1^].\n\nAs for patents in the field of Artificial Synapse, it's a bit more complex. There are several significant works in the field. For instance, there is research on artificial synapses based on polyimide with embedded graphene quantum dots. This memristor-type artificial synaptic device s